https://nanostring.com/products/cosmx-spatial-molecular-imager/ffpe-dataset/
Lung5_Rep1

https://nanostring-public-share.s3.us-west-2.amazonaws.com/SMI-Compressed/SMI-ReadMe.html

In [2]:
cd /content/drive/MyDrive/datasets/cosmx-smi-ffpe

/content/drive/MyDrive/datasets/cosmx-smi-ffpe


In [3]:
import pandas as pd

Build a feedforward network that predicts `target` from the dataset using the remaining attributes

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

In [5]:
class MyDataset(Dataset):
    def __init__(self, data_file, max_length=64, transform=None):
        dataset = pd.read_parquet(data_file)
        self.genes = dataset[dataset.columns[7:]]
        self.target = dataset['target']
        
        # train a label encoder here
        from sklearn import preprocessing
        le = preprocessing.LabelEncoder()
        self.target_labels=le.fit_transform(self.target)


    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        genes = self.genes.iloc[idx]
        target = self.target_labels[idx]
        # transformer target with label encoder here
        return np.array(genes,dtype=np.float32),target

TODO: train label encoder for target and then train/test the model

In [6]:
train_dataset=MyDataset(data_file="train.parquet")
test_dataset=MyDataset(data_file="test.parquet")

In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=64)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=64)


In [8]:
train_dataset[0]

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
        0., 1., 0., 0., 0., 2., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 

In [9]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(980, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 980),
            # nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=980, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=980, bias=True)
  )
)


In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


def train(train_dataloader, model, loss_fn, optimizer):
    size = len(train_dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(test_dataloader, model, loss_fn):
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in test_dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

 

1.   train and test nn model
2.   display accuracy



In [ ]:
for i in range(100):
    train(train_dataloader, model, loss_fn, optimizer)

loss: 17.462120  [    0/431477]
loss: 6.814611  [ 6400/431477]
loss: 6.842193  [12800/431477]
loss: 6.845583  [19200/431477]
loss: 6.816121  [25600/431477]
loss: 6.839427  [32000/431477]
loss: 6.849277  [38400/431477]
loss: 6.845391  [44800/431477]
loss: 6.754261  [51200/431477]
loss: 6.688385  [57600/431477]
loss: 6.745841  [64000/431477]
loss: 6.811589  [70400/431477]
loss: 6.653312  [76800/431477]
loss: 6.825336  [83200/431477]
loss: 6.744388  [89600/431477]
loss: 6.756376  [96000/431477]
loss: 6.745135  [102400/431477]
loss: 6.474960  [108800/431477]
loss: 6.780014  [115200/431477]
loss: 6.589030  [121600/431477]
loss: 6.665022  [128000/431477]
loss: 6.688357  [134400/431477]
loss: 6.672122  [140800/431477]
loss: 6.431937  [147200/431477]
loss: 6.478077  [153600/431477]
loss: 6.472515  [160000/431477]
loss: 6.352028  [166400/431477]
loss: 6.560683  [172800/431477]
loss: 6.309165  [179200/431477]
loss: 6.487651  [185600/431477]
loss: 6.424589  [192000/431477]
loss: 6.390934  [198400

In [ ]:
test(test_dataloader, model, loss_fn)

https://www.mdpi.com/2076-3417/11/2/808/htm#B9-applsci-11-00808
